<a href="https://colab.research.google.com/github/midnightripper/accuracy_improvement/blob/main/VAE-with_appropriate_bottlenecking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Prerequisite

In [2]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Reshape
from tensorflow.keras import backend as K

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Load the train and test data splits

In [4]:
GER_test = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_train.mat')['GER_train'])).transpose()
test_frame = pd.DataFrame(GER_test).sort_values(1)
train_frame = pd.DataFrame(GER_train).sort_values(1)

In [5]:
print(f"x_train shape: {GER_train.shape} - y_train shape: {GER_train.shape}")
print(f"x_test shape: {GER_test.shape} - y_test shape: {GER_test.shape}")

x_train shape: (6981, 21) - y_train shape: (6981, 21)
x_test shape: (6248, 21) - y_test shape: (6248, 21)


In [6]:
def get_data_labels(split):
    x = pd.DataFrame(split)
    labels = x[0].values.astype(np.uint8)
    del x[0],x[1]
    data = x.values 
    return data, labels
    
x_train, y_train = get_data_labels(train_frame.values.tolist())
x_test, y_test = get_data_labels(test_frame.values.tolist())

In [7]:
df = pd.DataFrame(x_train)
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [8]:
df2 = pd.DataFrame(x_test)
scaler = StandardScaler()
x_test = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns)

In [9]:
print(f"Size of the training set: {len(x_train)}")

Size of the training set: 6981


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(f"Size of the x training set: {len(x_train)}")
print(f"Size of the x validation set: {len(x_val)}")
print(f"Size of the y training set: {len(y_train)}")
print(f"Size of the y validation set: {len(y_val)}")

Size of the x training set: 5584
Size of the x validation set: 1397
Size of the y training set: 5584
Size of the y validation set: 1397


Early stopping to prevent over-fitting

In [11]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [12]:
def loss(outputs,inputs,z_log_var,z_mean):
  reconstruction_loss = K.mean(K.square(outputs - inputs), axis=-1)
  kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
  vae_loss = K.mean(reconstruction_loss + kl_loss)
  return vae_loss





# Function Calls

In [13]:
input_shape = (19,)

In [14]:
def sampling(args):
    z_mean, z_log_var = args
    eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1. , name='epsilon')
    z = z_mean + tf.exp(z_log_var / 2) * eps
    return z

In [15]:
inputs = keras.layers.Input(shape=input_shape, name='input')
x = keras.layers.Dense(32, activation='relu')(inputs)
x = keras.layers.Dense(16, activation='relu')(x)
z_mean = keras.layers.Dense(8, name='z_mean')(x)
z_log_var = keras.layers.Dense(8, name='z_log_var')(x)

# Decoder

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1 , name='epsilon')
    z = z_mean + tf.exp(z_log_var / 2) * eps
    return z


#Giving input to decoder and initialising shape
z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
encoder = keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')
#decoder
latent_inputs = keras.layers.Input(shape=(8), name='z_sampling')
x = keras.layers.Dense(16, activation='relu')(latent_inputs)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dense(19, activation='relu')(x)

# instantiate decoder model
decoder = keras.Model(latent_inputs, x, name='decoder')
outputs = decoder(encoder(inputs)[2])  # Select the Z value from outputs of the encoder
vae = keras.Model(inputs, outputs, name='vae')

autoencoder = Model(inputs, outputs)
autoencoder.add_loss(loss(outputs,inputs,z_log_var,z_mean))
autoencoder.compile(optimizer='adam')

history = autoencoder.fit(x_train, x_train,
                epochs=200,
                batch_size=128,
                shuffle=True,
                validation_data=(x_val,x_val),callbacks=[es])

Epoch 1/200
44/44 [==============================] - 2s 10ms/step - loss: 1.7894 - val_loss: 1.2813
Epoch 2/200
44/44 [==============================] - 0s 3ms/step - loss: 1.1382 - val_loss: 1.1134
Epoch 3/200
44/44 [==============================] - 0s 4ms/step - loss: 1.0536 - val_loss: 1.0686
Epoch 4/200
44/44 [==============================] - 0s 4ms/step - loss: 1.0242 - val_loss: 1.0511
Epoch 5/200
44/44 [==============================] - 0s 4ms/step - loss: 1.0125 - val_loss: 1.0423
Epoch 6/200
44/44 [==============================] - 0s 4ms/step - loss: 1.0047 - val_loss: 1.0379
Epoch 7/200
44/44 [==============================] - 0s 7ms/step - loss: 1.0024 - val_loss: 1.0347
Epoch 8/200
44/44 [==============================] - 0s 4ms/step - loss: 0.9987 - val_loss: 1.0329
Epoch 9/200
44/44 [==============================] - 0s 3ms/step - loss: 0.9977 - val_loss: 1.0316
Epoch 10/200
44/44 [==============================] - 0s 4ms/step - loss: 0.9942 - val_loss: 1.0307
Epoch 11

In [ ]:
encoder = Model(inputs, z)
encoder.trainable = False
encoder.summary()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from keras.layers import Input, Dense
from keras.models import Model
import seaborn as sns

In [ ]:
encoded_data = encoder.predict(x_train)
print(encoded_data.shape)
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(encoded_data)
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="Encoder T-SNE projection")

In [ ]:
encoded_data = autoencoder.predict(x_train)
print(encoded_data.shape)
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(encoded_data)
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="Autoencoder T-SNE projection")